# pip installs

In [ ]:
!pip install llvmpy
!pip install cython #==0.29.24
!pip install numba #==0.56.2
!pip install pandas #==1.3.4
!pip install networkx #==2.6.3
!pip install matplotlib #==3.4.3
!pip install ts2vg #==1.0.0
!pip install pytorch_lightning==1.6.5
!pip install tensorflow #==2.11.0
!pip install dvclive #==1.3.3
!pip install torchsummary #==1.5.1
!pip install torchmetrics==0.6.0

# pip from 31/1 forward

In [ ]:
!mamba install pytorch-cuda=11.6 -c pytorch -c conda-forge -c nvidia -y -q

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116 -q

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.1+cu116.html -q

In [ ]:
!pip install ts2vg -q

In [ ]:
!pip install pytorch_lightning==1.6.5 -q

In [ ]:
!mamba install tensorflow-gpu -y -q

In [ ]:
!mamba install -c conda-forge pyts -q -y #==0.12.0

In [ ]:
!pip install torchsummary -q

# Main code

In [3]:
"""
bellow are listed all nedded libraries
"""

import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import warnings
import sklearn
import os.path as osp
import matplotlib.pyplot as plt
import ts2vg
import pytorch_lightning as pl
import tensorflow as tf
import datetime

from pyts.image import MarkovTransitionField

from torch.nn import Linear, CrossEntropyLoss, BCEWithLogitsLoss
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool, global_add_pool, global_max_pool, ChebConv, global_sort_pool
from torch_geometric.loader import DataLoader
from torch.nn import Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv, GINEConv, GATv2Conv, GATConv

from tqdm import tqdm

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.manifold import TSNE

from torchvision import transforms

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import ProgressBarBase
from pytorch_lightning.loggers import TensorBoardLogger
#from pytorch_lightning.loggers import CSVLogger
#from dvclive.lightning import DVCLiveLogger

#from torchsummary import summary

from ts2vg import NaturalVG
from ts2vg import HorizontalVG

In [4]:
"""

in this bracket of code is located the conversion of time series to different graph neural networks (Markov transition field (MTF) and Visibility graphs (VG)) 
with different lengths (uncut(300), cut and random(where every sample has a different length between 300 and 50)).

create graph is called when we have a time series that needs to be converted into a graph. I

"""
def create_graph():
    warnings.filterwarnings("ignore")

    # preparation for un/cut graphs
    if len_type == "un/cut":
    
        df = pd.read_csv(path_main)  
        del df['Unnamed: 0']
        df.index, df.columns = [range(df.index.size), range(df.columns.size)]
        length_rss = int((df.columns.stop-2)/2)
        
        X = df.loc[:,df.columns[:length_rss]].to_numpy() # x values for every sample
        #X = np.round_(X,1)
        Y = df[length_rss+1].to_numpy(dtype=np.uint8) # types of anomalies
        X_mask = df.loc[:,df.columns[length_rss+2:]].to_numpy() # binary location of anomalies
        
        if graph_type in ("MTF", "vis_on_MTF", "MTF_on_vis"):
            MTF = MarkovTransitionField(n_bins=length_rss)
            X_gaf_MTF = MTF.fit_transform(X)
        
    # preparation for random graphs
    elif len_type == "random":
        dataset_rss = np.load(path_main, allow_pickle=True)['arr_0']
        dataset_properties = np.load(path_properties, allow_pickle=True)['arr_0']
        dataset_mask = np.load(path_mask, allow_pickle=True)['arr_0']

        for i in range(len(dataset_properties)):
            dataset_properties[i,1] = int(dataset_properties[i,1])
        
        X = dataset_rss # x values for every sample
        X_mask = dataset_mask # binary location of anomalies
        Y = dataset_properties[:,2] # types of anomalies
        Y_len = dataset_properties[:,0] # length of every sample
        
        if graph_type in ("MTF", "vis_on_MTF", "MTF_on_vis"):
            X_gaf_MTF = []
            for i in range(len(Y_len)):
                MTF = MarkovTransitionField(n_bins=Y_len[i])
                X_gaf_MTF_temp = MTF.fit_transform(X[i].reshape(1, -1))
                X_gaf_MTF.append(X_gaf_MTF_temp[0])
    
    # output will contain all graphs 
    output = []
    
    # setting class_weights for graph
    global class_weights
    class_weights = torch.tensor(class_weight.compute_class_weight(class_weight='balanced',
                                                                   classes=np.unique(Y),
                                                                   y=Y))
    def vis_matrix(X_current, vis_type_local):
        if vis_type_local == "natural":
            g = NaturalVG(weighted=vis_distance)
        elif vis_type_local == "horizontal":
            g = HorizontalVG(weighted=vis_distance)
            
        g.build(X_current)

        adj_mat_vis = np.zeros([len(X_current),len(X_current)], dtype='float')
        for i in range(len(g.edges)):
            x, y, q =g.edges[i]
            adj_mat_vis[x,y] = q
            if vis_edge_type == "undirected":
                adj_mat_vis[y,x] = q
        
        return adj_mat_vis
    
    # functions for creating edge index and edge weight for a given matrix
    def adjToEdgidx(adj_mat):
        #function for visibility and MTF matrixes
        edge_index = torch.from_numpy(adj_mat).nonzero().t().contiguous()
        row, col = edge_index
        edge_weight = adj_mat[row, col]
        return edge_index,edge_weight
    
    def adjToEdgidx_dual_VG(X_current):
        #function for joined visibility and MTF matrixes
        pos_adj_mat_vis = vis_matrix(X_current, vis_type)
        neg_adj_mat_vis = vis_matrix(-X_current, vis_type)
            
        edge_index = torch.from_numpy(pos_adj_mat_vis+neg_adj_mat_vis).nonzero().t().contiguous()
        
        #join two edge_weight arrays
        row, col = edge_index
        edge_weight = np.zeros([len(row),2], dtype='float')
        edge_weight[:,0] = pos_adj_mat_vis[row, col]
        edge_weight[:,1] = neg_adj_mat_vis[row, col]
        return edge_index, edge_weight
        
    def define_mask(i):
        if classif == "graph": # for graph classification
            return torch.tensor(Y[i], dtype=torch.long)
        elif classif == "node":# for node classification 
            return torch.unsqueeze(torch.tensor(X_mask[i], dtype=torch.double),1)
        
    
    if graph_type == "MTF":  
        for i, j in enumerate(X_gaf_MTF):
            edge_index, edge_weight = adjToEdgidx(j)
            y_mask = define_mask(i)
            #Into Data save node values "x", edge index from adjacency matrix and edge features/attributes, finally label
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1), y=y_mask))
    
    elif graph_type == "vis":
        for i in range(len(X)):
            edge_index, edge_weight = adjToEdgidx(vis_matrix(X[i],vis_type))
            y_mask = define_mask(i)
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=torch.tensor(edge_index, dtype=torch.int64), edge_attr=torch.unsqueeze(torch.tensor(edge_weight, dtype=torch.double),1),y=y_mask))    
    
    elif graph_type == "dual_VG":
        for i in range(len(X)):
            edge_index, edge_weight = adjToEdgidx_dual_VG(X[i])
            y_mask = define_mask(i) 
            output.append(Data(x=torch.unsqueeze(torch.tensor(X[i], dtype=torch.double),1), edge_index=edge_index, edge_attr=torch.tensor(edge_weight, dtype=torch.double), y=y_mask))    
    
    return output    


In [5]:
"""
in this function we use GINE for graph classification and Net for node classidication using Pytorch Lightning for the graphs we created by using the create_graph function.
This function also incorporates callbacks for easier and faster result agregation.
"""

# class for node classification            
class GINE(pl.LightningModule):
    def __init__(self):
        super(GINE, self).__init__()
        
        if graph_type in ("MTF", "vis"):
            edge_dim = 1
        elif graph_type in ("MTF_on_vis", "vis_on_MTF", "double_VG", "dual_VG"):
            edge_dim = 2
            
        dim_h = 32
    
        self.conv1 = GINEConv(
            Sequential(Linear(dim_h, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv2 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv3 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv4 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        self.conv5 = GINEConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()), edge_dim=edge_dim)
        
        
        self.lin1 = Linear(dim_h*5, dim_h*5)
        self.lin2 = Linear(dim_h*5, 5)
    
    
    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr, data.batch
        
        # Node embeddings 
        h1 = self.conv1(x, edge_index, edge_attr=edge_weight)
        h2 = self.conv2(h1, edge_index, edge_attr=edge_weight)
        h3 = self.conv3(h2, edge_index, edge_attr=edge_weight)
        h4 = self.conv4(h3, edge_index, edge_attr=edge_weight)
        h5 = self.conv5(h4, edge_index, edge_attr=edge_weight)
        
        # Graph-level readout
        
        h1 = global_max_pool(h1, batch)
        h2 = global_max_pool(h2, batch)
        h3 = global_max_pool(h3, batch)
        h4 = global_max_pool(h4, batch)
        h5 = global_max_pool(h5, batch)
        
        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3, h4, h5), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)
        
        return h
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
                     
        out = model(train_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        train_loss = loss_function(out, train_batch.y)
        
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        val_loss = loss_function(out, val_batch.y)
        
        pred = out.argmax(-1)
        correct=out.argmax(dim=1).eq(val_batch.y).sum().item()
        total=len(val_batch.y)
        val_label = val_batch.y
        accuracy = (pred == val_label).sum() / pred.shape[0]
        
        logs={"train_loss": val_loss}
        batch_dictionary={"loss": val_loss, "log": logs, "correct": correct, "total": total}
        
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
        
    
    def test_step(self, test_batch, batch_idx):
        out = model(test_batch)
        loss_function = CrossEntropyLoss(weight=class_weights).to(device)
        test_loss = loss_function(out, test_batch.y)
        
        pred = out.argmax(-1)
        test_label = test_batch.y
        accuracy = (pred == test_label).sum() / pred.shape[0]
        self.log("test_true", test_label)
        self.log("test_pred", pred)
        self.log("test_acc", accuracy)
        return pred, test_label
        
    def test_epoch_end(self, outputs):
        #this function gives us in the outputs all acumulated pred and test_labels we returned in test_step
        #we transform the pred and test_label into a shape that the classification report can read
        true_array=[]
        pred_array = []
        for i in range(len(outputs)):
            true_array = np.append(true_array,outputs[i][1].cpu().numpy())
            pred_array = np.append(pred_array,outputs[i][0].cpu().numpy())            
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))
        return pred_array, true_array
    
#---------------------------------------------------------------------------------
# class for node classification
class Net(pl.LightningModule):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = GATConv(1, 32, heads=4)
        self.lin1 = torch.nn.Linear(1, 4 * 32)
        self.conv2 = GATConv(4 * 32, 32, heads=4)
        self.lin2 = torch.nn.Linear(4 * 32, 4 * 32)
        self.conv3 = GATConv(4 * 32, 1, heads=6,concat=False)
        self.lin3 = torch.nn.Linear(4 * 32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.elu(self.conv1(x, edge_index) + self.lin1(x))
        x = F.elu(self.conv2(x, edge_index) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=5e-4)
        return optimizer
    


    def training_step(self, train_batch, batch_idx):        
        out = model(train_batch)
        loss_function = BCEWithLogitsLoss().to(device) #weight=class_weights
        
        train_loss = loss_function(out, train_batch.y)
        correct=out.argmax(dim=1).eq(train_batch.y).sum().item()
        logs={"train_loss": train_loss}
        total=len(train_batch.y)
        
        batch_dictionary={"loss": train_loss, "log": logs, "correct": correct, "total": total}
        
        return train_loss
    
    
    def validation_step(self, val_batch, batch_idx):
      
        out = model(val_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        val_loss = loss_function(out, val_batch.y)
        
        ys, preds = [], []
        val_label = val_batch.y.cpu()
        ys.append(val_batch.y)
        preds.append((out > 0).float().cpu())     
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == val_label).sum() / pred.shape[0]
    
        self.log("val_loss", val_loss)
        self.log("val_acc", accuracy)
    
    def test_step(self, test_batch, batch_idx):
        # this is the test loop
        out = model(test_batch)
        loss_function = BCEWithLogitsLoss().to(device)
        test_loss = loss_function(out, test_batch.y)
        
        ys, preds = [], []
        test_label = test_batch.y.cpu()
        ys.append(test_batch.y)
        preds.append((out > 0).float().cpu())
        
        y, pred = torch.cat(ys, dim=0), torch.cat(preds, dim=0)
        accuracy = (pred == test_label).sum() / pred.shape[0]
        
        self.log("test_acc", accuracy)
        return pred, y
        
    def test_epoch_end(self, outputs):
        #this function gives us in the outputs all acumulated pred and test_labels we returned in test_step
        #we transform the pred and test_label into a shape that the classification report can read
        global true_array, pred_array
        true_array=[outputs[i][1].cpu().numpy() for i in range(len(outputs))]
        pred_array = [outputs[i][0].cpu().numpy() for i in range(len(outputs))]
        print(confusion_matrix(true_array, pred_array))
        print(classification_report(true_array, pred_array))
        print("pred_array ",pred_array)

        
def main():
    if check_for_missclick() != True:
        print("one of the main parameters does not match")
    else:
        global model, test_loader, device
        
        early_stop = EarlyStopping(monitor='val_acc',patience=patience, strict=False,verbose=False, mode='max')
        val_checkpoint_acc = ModelCheckpoint(filename="max_acc-{epoch}-{step}-{val_acc:.3f}", monitor = "val_acc", mode="max")
        val_checkpoint_best_loss = ModelCheckpoint(filename="best_loss", monitor = "val_loss", mode="max")
        val_checkpoint_best_acc = ModelCheckpoint(filename="best_acc", monitor = "val_acc", mode="max")
        val_checkpoint_loss = ModelCheckpoint(filename="min_loss-{epoch}-{step}-{val_loss:.3f}", monitor = "val_loss", mode="min")
        latest_checkpoint = ModelCheckpoint(filename="latest-{epoch}-{step}", monitor = "step", mode="max",every_n_train_steps = 500,save_top_k = 1)
        #batchsizefinder = BatchSizeFinder(mode='power', steps_per_trial=3, init_val=2, max_trials=25, batch_arg_name='batch_size')
        #lr_finder = FineTuneLearningRateFinder(milestones=(5,10))
        tb_logger = TensorBoardLogger(save_file, name=name_of_save) # where the model saves the callbacks
        #dvclive_logger = DVCLiveLogger()
        
        torch.manual_seed(SEED)
        print(SEED)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        output = generate_output() # creates dataset with graphs  
        
        train_size = int(0.8 * len(output))
        Temp_size = len(output) - train_size
        val_size = int(0.2*Temp_size)
        test_size = Temp_size - val_size
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])
                
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=int(batch_size/2), shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        # mode
        if classif == "graph":
            model = GINE().double()#.to(device) #GINE is for graph classification
        elif classif == "node":
            model = Net().double()#.to(device) #net is for node classification
            
        #training
        
        trainer = pl.Trainer(logger=tb_logger, max_epochs = range_epoch, callbacks=[val_checkpoint_best_loss, val_checkpoint_best_acc, latest_checkpoint, val_checkpoint_acc,val_checkpoint_loss,early_stop],accelerator='gpu',devices=1)
        trainer.fit(model, train_loader, val_loader)
        
# it automaticly checks number of saves in folder and runs all of them in order
def my_test_loop():
    if check_for_missclick() != True:
        print("one of the main parameters does not match")
    else:
        import os, sys
        versions = os.listdir("Models/"+version)
        len_ver = len(versions)
        SEED_temp = SEED
        for i in range(len_ver):

            global device, model

            torch.manual_seed(SEED_temp)
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            output = generate_output() # creates dataset with graphs

            train_size = int(0.8 * len(output))
            Temp_size = len(output) - train_size
            val_size = int(0.2*Temp_size)
            test_size = Temp_size - val_size
            train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(output, [train_size, val_size, test_size])

            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
            
            if classif == "graph":
                model = GINE.load_from_checkpoint("Models/"+version+"/version_" + str(i) + "/checkpoints/" + monitor_test).double()
            elif classif == "node":
                model = Net.load_from_checkpoint("Models/"+version+"/version_" + str(i) + "/checkpoints/" + monitor_test).double()
            
            trainer = pl.Trainer(accelerator='gpu',devices=1)
            trainer.test(model, test_loader)
            # checks if version is the same as seed(just to bo sure)
            print("version_"+str(i))
            print(SEED_temp)
            SEED_temp += 1
#------------------------------------------------------------


#main parametrs
graph_type = "vis" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
cut_len = '0' # 0 if we arent doing cut by other lengths than 300 or random 
vis_type = "natural" #"natural", "horizontal"
vis_distance = 'distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"
#paths
path_main = "dataset_100_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "dataset_properties.npz"  # path to properties used for random 
path_mask = "dataset_mask.npz" # path to mask dataset used for random

# params for 
monitor_test = "best_acc.ckpt"
learning_rate = 0.01
batch_size = 64*2
range_epoch = 500 #set length of epoch
save_file="Models"
name_of_save = "100"
#-------------------------------------------------------------

In [6]:
#defines the anomaly type of node classification prediction
def anomaly_type(my_model):
    #I know that the program very often predicts, if model has all 300 samples, that the first 3 are anomalies, and that is not true so ...
    if len(my_model) == 300:
        my_model[0] = 0
        my_model[1] = 0
        my_model[2] = 0
    
    if len(my_model)%2 != 0:
        my_model = np.append(0,my_model)
    par1 = int(len(my_model)/2)
    
    #check if anywhere in array there is an anomaly, otherwise predict this is a No_anomaly
    if any(my_model) == 1:
        
        mm1 = [all(i) for i in my_model.reshape(par1, -1)]
        mm2 = [all(i) for i in my_model[1:-1].reshape(par1, -1)]
        #check if anywhere in the array there are two 1 gruped together, otherwise predict this is a InstaD
        if any(mm1) == 1 and any(mm2) == 1:
        
            #check if last in the array is 1 (this tells me if the anomaly ends or not), otherwise predict this is a SuddenR
            if my_model[-1] == 1:
                
                mm3 = [all(i) for i in my_model.reshape(2, -1)]
                #you can't differentiate SlowD from SuddenD other than that that SlowD starts very early in the array, so if second half of the array is all 1 predict Slow, otherwise predict this is a SuddenD
                if any(mm3) == 1:
                    
                    return "SlowD"
                return "SuddenD"
            return "SuddenR"
        return "InstaD"
    return "No_anomaly"

#defines where the anomalies are (where they start and end) and put it in a text
def Anomaly_loc(pred_type,pred_pred):
    W_list = []
    for i in range(len(pred_type)):
        
        result = [j for j, x in enumerate(pred_model[i]) if x]
        
        if pred_type[i] == "No_anomaly":
            W = "There is no anomaly"
            
        elif pred_type[i] == "InstaD":
            result_str = [str(a) for a in result]
            W = "Anomaly type is Instant Degredation and the anomalies are located at times "+', '.join(result_str)
            
        elif pred_type[i] == "SuddenR":
            W = "Anomaly type is Sudden Recovery and the anomalies are located at times from " + str(result[0]) + " to " + str (result[-1])
            
        elif pred_type[i] == "SuddenD":
            W = "Anomaly type is Sudden Degredation and the anomalies are located at times from " + str(result[0]) + " and do not stop"
        
        elif pred_type[i] == "SlowD":
            W = "Anomaly type is Slow Degredation and the anomalies are located at times from " + str(result[0]) + " and do not stop"
        W_list = np.append(W_list, W)

    return W_list

In [7]:
"""
this definition keeps the last output so we dont have to generate another if all parameters are the same as before, usefull for when you change the modell a lot but not the graph itself
"""
global temp_repeat
#we have 9 parameters to remember
temp_repeat=['']*9
def generate_output():
    if temp_repeat[0] != graph_type or temp_repeat[1] != classif or temp_repeat[2] != len_type or temp_repeat[3] != path_main or temp_repeat[4] != path_properties or temp_repeat[5] != path_mask or temp_repeat[6] != vis_type or temp_repeat[7] != vis_distance or temp_repeat[8] != vis_edge_type:
    
        global output
        output = create_graph()
        
    #remembering the last known parameter
    temp_repeat[0] = graph_type 
    temp_repeat[1] = classif
    temp_repeat[2] = len_type
    temp_repeat[3] = path_main
    temp_repeat[4] = path_properties
    temp_repeat[5] = path_mask
    temp_repeat[6] = vis_type
    temp_repeat[7] = vis_distance
    temp_repeat[8] = vis_edge_type
    
    print(temp_repeat[1] + " classification on " + temp_repeat[2] + " time series using " + temp_repeat[0] + " graphs" )
    return output
#if we write a parameter that is not known to the function this def will stop it faster beffore you get an error half through the graph generation 
def check_for_missclick():
    return graph_type in ("vis", "MTF", "MTF_on_vis", "vis_on_MTF","double_VG","dual_VG") and classif in ("graph", "node") and len_type in ("un/cut", "random") and vis_type in ("natural", "horizontal") and vis_distance in ('slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance') and vis_edge_type in ("undirected", "directed")

# Training

In [8]:
"""
Seed numbers for every instance of dataset (they add up (VG + graph + random))
"""

Dict = {
    #graph_type
        "MTF" : 0,  
        "vis" : 1000, 
        "MTF_on_VG" : 2000,  
        "VG_on_MTF" : 3000,
        "dual_VG" : 0,
    #classif
        "graph" : 0,  
        "node" : 100,  
    #len_type
        "un/cut" : 0,  
        "random" : 10,  
    #cut_len
        "0" : 0,  
        "50" : 20,  
        "100" : 30,  
        "150" : 40,  
        "200" : 50,  
        "250" : 60,  
        "25" : 70,
    #parameters for visibility
        "natural" : 0,  
        "horizontal" : 4000,  
    
        "distance" : 0,
        "v_distance" : 100000,  
        "abs_v_distance" : 200000,  
        "slope" : 300000,
        "abs_slope" : 400000,
        
        "undirected":0,
        "directed" : 5000
}

for key, value in Dict.items():
    print(key, ' : ', value)

MTF  :  0
vis  :  1000
MTF_on_VG  :  2000
VG_on_MTF  :  3000
dual_VG  :  0
graph  :  0
node  :  100
un/cut  :  0
random  :  10
0  :  0
50  :  20
100  :  30
150  :  40
200  :  50
250  :  60
25  :  70
natural  :  0
horizontal  :  4000
distance  :  0
v_distance  :  100000
abs_v_distance  :  200000
slope  :  300000
abs_slope  :  400000
undirected  :  0
directed  :  5000


In [ ]:
graph_type = "vis" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", "dual_VG", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
cut_len = '0' # 0 if we arent doing cut by other lengths than 300 or random 
vis_type = "natural" #"natural", "horizontal"
vis_distance = 'distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"

path_main = "datasets/dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask.npz" # path to mask dataset used for random

SEED = int(Dict[graph_type]+Dict[classif]+Dict[len_type]+Dict[cut_len]+Dict[vis_distance]+Dict[vis_type]+Dict[vis_edge_type])
learning_rate = 0.0005
batch_size = 64
range_epoch = 1000 #set length of epoch
save_file="test"
name_of_save = "VG_dual_v_abs_compact_" + str(SEED)
patience = 400
analysis = True
#-------------------------------------------------------------
for i in range(1): 
    SEED += i
    main()

In [ ]:
graph_type = "dual_VG" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", "dual_VG", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
cut_len = '0' # 0 if we arent doing cut by other lengths than 300 or random 
vis_type = "horizontal" #"natural", "horizontal"
vis_distance = 'abs_v_distance'#'slope', 'abs_slope','distance','h_distance','v_distance','abs_v_distance',
vis_edge_type = "directed" #"undirected", "directed"


path_main = "datasets/dataset_uncut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
path_properties = "datasets/dataset_properties.npz"  # path to properties used for random 
path_mask = "datasets/dataset_mask.npz" # path to mask dataset used for random

SEED = int(Dict[graph_type]+Dict[classif]+Dict[len_type]+Dict[cut_len]+Dict[vis_distance]+Dict[vis_type]+Dict[vis_edge_type])
learning_rate = 0.0005
batch_size = 64
range_epoch = 1000 #set length of epoch
save_file="Models_not_faulty"
name_of_save = "HVG_dual_v_abs_" + str(SEED)
patience = 400
analysis = True
#-------------------------------------------------------------
for i in range(5): 
    SEED += i
    main()

# testing

In [ ]:
version = "VG_50"
monitor_test = "best_acc.ckpt"
graph_type = "vis" #"vis", "MTF", "MTF_on_vis", "vis_on_MTF", graph topology
classif = "graph" #"graph", "node", set the type of classification
len_type = "un/cut" #"un/cut", "random", the shape of data used in later paths 
path_main = "datasets/dataset_50_cut.csv" #"dataset_uncut.csv", "dataset_cut.csv", "dataset_rss.npz", paths used for cut/uncut/random dataset
SEED = 1020

my_test_loop()